In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential #type: ignore
from tensorflow.keras.layers import LSTM, Dense #type: ignore
from datetime import timedelta
import json


In [ ]:
def train_model(stockticker, predictionDay):
    # Download historical stock data using the Yahoo Finance API
    maindf = yf.download(stockticker)
    maindf = maindf.reset_index()
    closedf = maindf[['Date', 'Close']]

    # Filter data for dates after February 15, 2021 and create a copy
    closedf = closedf[closedf['Date'] > '2021-02-15']
    close_stock = closedf.copy()

    # Remove the date column and scale the close prices between 0 and 1
    last_date = closedf['Date'].iloc[-1]  # Save the last date
    del closedf['Date']
    scaler = MinMaxScaler(feature_range=(0, 1))
    closedf = scaler.fit_transform(np.array(closedf).reshape(-1, 1))

    # Split the data into training and test sets
    training_size = int(len(closedf) * 0.60)
    test_size = len(closedf) - training_size
    train_data, test_data = closedf[0:training_size, :], closedf[training_size:len(closedf), :1]

    # Function to create a dataset with X (features) and Y (target) sets
    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset) - time_step - 1):
            a = dataset[i:(i + time_step), 0]
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return np.array(dataX), np.array(dataY)

    # Define the number of previous days to use for predicting the next value
    time_step = 15
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    # Reshape input to be [samples, time steps, features] for LSTM
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # Define and compile the LSTM model
    model = Sequential()
    model.add(LSTM(10, input_shape=(None, 1), activation="relu"))
    model.add(Dense(1))
    model.compile(loss="mean_squared_error", optimizer="adam")

    # Fit the model using the training data
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=32, verbose=1)

    # Prepare input for future prediction
    x_input = test_data[len(test_data) - time_step:].reshape(1, -1)
    temp_input = list(x_input[0])

    # Predict future values
    predictions = {}
    for i in range(predictionDay):
        x_input = np.array(temp_input[-time_step:]).reshape(1, -1)
        x_input = x_input.reshape((1, time_step, 1))
        yhat = model.predict(x_input, verbose=0)
        yhat_real_value = scaler.inverse_transform(yhat)
        temp_input.extend(yhat[0].tolist())

        # Save the predicted values with dates
        predicted_date = (last_date + timedelta(days=i + 1)).strftime('%Y-%m-%d')
        predictions[predicted_date] = float(yhat_real_value[0][0])

    # Save predictions to a JSON file
    with open('predictions.json', 'w') as json_file:
        json.dump(predictions, json_file, indent=4)

    return predictions


In [ ]:
train_model("BTC-USD",60)

[*********************100%%**********************]  1 of 1 completed


Epoch 1/200
22/22 [==============================] - 2s 20ms/step - loss: 0.3385 - val_loss: 0.2368
Epoch 2/200
22/22 [==============================] - 0s 10ms/step - loss: 0.2493 - val_loss: 0.1724
Epoch 3/200
22/22 [==============================] - 0s 10ms/step - loss: 0.1807 - val_loss: 0.1207
Epoch 4/200
22/22 [==============================] - 0s 12ms/step - loss: 0.1194 - val_loss: 0.0731
Epoch 5/200
22/22 [==============================] - 0s 12ms/step - loss: 0.0583 - val_loss: 0.0214
Epoch 6/200
22/22 [==============================] - 0s 12ms/step - loss: 0.0103 - val_loss: 0.0029
Epoch 7/200
22/22 [==============================] - 0s 12ms/step - loss: 0.0033 - val_loss: 0.0032
Epoch 8/200
22/22 [==============================] - 0s 11ms/step - loss: 0.0027 - val_loss: 0.0023
Epoch 9/200
22/22 [==============================] - 0s 12ms/step - loss: 0.0023 - val_loss: 0.0017
Epoch 10/200
22/22 [==============================] - 0s 14ms/step - loss: 0.0020 - val_loss: 0.0015

{'2024-04-16': 63687.51171875,
 '2024-04-17': 63821.21875,
 '2024-04-18': 64200.5234375,
 '2024-04-19': 64617.99609375,
 '2024-04-20': 65058.046875,
 '2024-04-21': 65420.64453125,
 '2024-04-22': 65701.3046875,
 '2024-04-23': 65899.015625,
 '2024-04-24': 66032.0078125,
 '2024-04-25': 66114.5625,
 '2024-04-26': 66166.4140625,
 '2024-04-27': 66207.7578125,
 '2024-04-28': 66256.9609375,
 '2024-04-29': 66303.8671875,
 '2024-04-30': 66357.8828125,
 '2024-05-01': 66409.6015625,
 '2024-05-02': 66467.609375,
 '2024-05-03': 66528.5234375,
 '2024-05-04': 66591.671875,
 '2024-05-05': 66655.7265625,
 '2024-05-06': 66719.5703125,
 '2024-05-07': 66782.2265625,
 '2024-05-08': 66843.171875,
 '2024-05-09': 66902.15625,
 '2024-05-10': 66959.1484375,
 '2024-05-11': 67014.25,
 '2024-05-12': 67067.6640625,
 '2024-05-13': 67119.5625,
 '2024-05-14': 67170.1796875,
 '2024-05-15': 67219.6484375,
 '2024-05-16': 67268.0859375,
 '2024-05-17': 67315.59375,
 '2024-05-18': 67362.203125,
 '2024-05-19': 67407.9140625,
